In [ ]:

input_sdate = "2023 08 20"
input_edate = "2023 08 25"
geometry = [112.70505, -44.52755, 154.38241, -11.29524]

In [ ]:

%matplotlib inline
import matplotlib.pyplot as plt
import netCDF4 as nc4
import xarray as xr
import fsspec
import numpy as np
import xarray as xr
import planetary_computer
import pystac_client
import geopandas as gpd
import pandas as pd
import cartopy.crs as ccrs
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime, time

In [ ]:
# Initialize PySTAC client for data query
planetary_computer.set_subscription_key("c27669c4bdec434d804e2bd738cb16fc")
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [ ]:
# Function to convert date format 
def convert_format_date(input_date):
    date_obj = datetime.strptime(input_date, "%Y %m %d")
    formatted_date = date_obj.strftime("%Y-%m-%d")
    return formatted_date

# Convert user start date format
input_sdate = convert_format_date(input_sdate)

# Convert user end date format
input_edate = convert_format_date(input_edate)

date_period = input_sdate + "/" + input_edate 
date_period

In [ ]:
search = catalog.search(
    collections="sentinel-5p-l2-netcdf",
    bbox = geometry,
    datetime=input_sdate + "/" + input_edate,
    query={"s5p:processing_mode": {"eq": "OFFL"}, "s5p:product_name": {"eq": "ch4"}},
)
items = search.item_collection()

print(len(items))

In [ ]:
#f1 = fsspec.open(items[0].assets['ch4'].href).open()
#f1
item_links = [item.assets['ch4'].href for item in items]
item_links

f = fsspec.open_files(item_links)
f = [file.open() for file in f]

ds = xr.open_mfdataset(f, group="PRODUCT", engine="h5netcdf", concat_dim='t', combine='nested') 
ds


In [ ]:
varname = "methane_mixing_ratio_bias_corrected"
data = ds[varname]
#print(data.shape)
vmin, vmax = np.nanpercentile(data, [1, 99])
#data.plot()
# methane product (NaN locations are transparent)
lon = ds["longitude"].values.squeeze()
lat = ds["latitude"].values.squeeze()
methane = data.values
#print(lon)

In [ ]:
sliced_lat = lat[0, 0, :, :]
#print(sliced_lat)
sliced_lon = lon[0, 0, :, :]
#print(sliced_lon)
sliced_methane = methane[0, 0, :, :]
#sliced_lat[np.isnan(sliced_lat)] = 0
#sliced_lon[np.isnan(sliced_lon)] = 0


#print("Shape of lat :", sliced_lat.shape)
#print("Shape of lon:", sliced_lon.shape)
#print("shape of methane: ", sliced_methane.shape)




print("NaN in lon:", np.isnan(sliced_lon).any())
print("NaN in lat:", np.isnan(sliced_lat).any())
print("NaN in methane:", np.isnan(sliced_methane).any())

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres')) # Get geopandas in-built naturalearth_lowres dataset
continent_borders = world.dissolve(by='continent')

fig, ax = plt.subplots(1, 1, figsize=(12, 8), subplot_kw={'projection': ccrs.PlateCarree()})

continent_borders.boundary.plot(ax=ax, linewidth=1, color='black')

ax.set_title("")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")




ax.coastlines()
ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, alpha=0.5, linestyle="--")
ax.set_extent(geometry, crs=ccrs.PlateCarree())
norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
colormesh = ax.pcolormesh (sliced_lat, sliced_lon, sliced_methane, cmap = "Spectral", norm = norm)
fig.colorbar(colormesh, pad=0.05, shrink=0.35, label="methane (mol/m2)")
plt.show()